In [1]:
# Dependencies
import requests
import matplotlib.pyplot as plt
import pandas as pd
from api_keys import weather_api_key
from citipy import citipy
import numpy as np 
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [2]:
#set ranges of lat and longs

lat_range = (-90, 90)
long_range = (-180, 180)
latitudes = np.random.uniform(lat_range[0], lat_range[1], size = 1500)
longitudes = np.random.uniform(long_range[0], long_range[1], size = 1500)

lat_long = zip(latitudes, longitudes)

In [3]:
#500 random cities
#Latitude values range from -90° to +90°. Longitude range from -180° to 180°

cities = []
for lat_lng in lat_long:
    city = citipy.nearest_city(lat_lng[0],lat_lng[1])
    if city.city_name not in cities:    
        cities.append(city.city_name)
    
    


In [4]:
len(cities)

633

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather"

parameters = {
    "appid": weather_api_key,
    "units": "imperial"
}

In [6]:
parameters["q"]=cities[0]
r = requests.get(url, parameters).json()
print(r)

{'coord': {'lon': 45.064, 'lat': 9.4092}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 80.78, 'feels_like': 70.88, 'temp_min': 80.78, 'temp_max': 80.78, 'pressure': 1016, 'humidity': 16, 'sea_level': 1016, 'grnd_level': 902}, 'visibility': 10000, 'wind': {'speed': 10.83, 'deg': 71}, 'clouds': {'all': 0}, 'dt': 1611820113, 'sys': {'country': 'SO', 'sunrise': 1611804090, 'sunset': 1611846219}, 'timezone': 10800, 'id': 53372, 'name': 'Oodweyne', 'cod': 200}


In [7]:
print("Beginning Data Retrieval")   
print("-----------------------------")

city_data = []

#look back at weatherpy print records and count when reviewing hw 
record_count = 1
set_count = 1

for i, city in enumerate(cities):
    
    if (i%50==0 and i>=50):
        set_count += 1 
        record_count = 0 
        
    url = "http://api.openweathermap.org/data/2.5/weather"

    parameters = {
    "appid": weather_api_key,
    "units": "imperial"
    }  
    
    parameters["q"]=city
   # print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1
    try:
        r = requests.get(url, parameters).json()
        

        city_data.append({
            "name":r["name"],
            "Latitude": r["coord"]["lat"],
            "Longitude": r["coord"]["lon"],
            "Temp_Max": r["main"]["temp_max"],
            "Humidity": r["main"]["humidity"],
            "Cloudiness": r["clouds"]["all"],
            "Wind Speed": r["wind"]["speed"],
            "Country": r["sys"]["country"],
            "Date": r["dt"]
        
        })
        
    except:
       # print("City not found. Skipping...")
        pass
#print("-----------------------------")
#print("Data Retrieval Complete")
#print("-----------------------------")


Beginning Data Retrieval
-----------------------------


In [11]:
df = pd.DataFrame(city_data)
df
df.to_csv("vacay_cities.csv")
df

,name,Latitude,Longitude,Temp_Max,Humidity,Cloudiness,Wind Speed,Country,Date
0,Oodweyne,9.4092,45.0640,80.78,16,0,10.83,SO,1611820113
1,Mataura,-46.1927,168.8643,48.99,80,76,10.00,NZ,1611820113
2,Diamantino,-14.4086,-56.4461,71.42,85,100,1.41,BR,1611820114
3,Arraial do Cabo,-22.9661,-42.0278,75.67,84,100,23.51,BR,1611820114
4,Saskylakh,71.9167,114.0833,-44.28,78,48,3.24,RU,1611820114
...,...,...,...,...,...,...,...,...,...
577,Hambantota,6.1241,81.1185,89.60,66,90,10.36,LK,1611820204
578,Oskemen,49.9565,82.6119,23.00,74,90,8.95,KZ,1611820204
579,Aitape,-3.1374,142.3475,79.34,85,100,5.48,PG,1611820204
580,Myrtle Grove,30.4210,-87.3075,50.00,66,1,10.36,US,1611820204


In [14]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

# * A max temperature lower than 80 degrees but higher than 70.
# * Wind speed less than 10 mph.
# * Zero cloudiness.
# * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable numb

df_vacay_filter = df[(df["Temp_Max"] <80) 
                & (df["Temp_Max"] >70)
                & (df["Wind Speed"] <10)
                & (df["Cloudiness"] == 0)]
df_vacay_filter
        
#* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


,name,Latitude,Longitude,Temp_Max,Humidity,Cloudiness,Wind Speed,Country,Date
61,Hermanus,-34.4187,19.2345,72.00,55,0,1.01,ZA,1611820090
113,Ashta,23.0167,76.7167,78.35,23,0,5.41,IN,1611820131
170,Hong Kong,22.2855,114.1577,73.00,63,0,9.31,HK,1611819952
227,Bubaque,11.2833,-15.8333,77.92,54,0,6.11,GW,1611820149
242,Bandar-e Lengeh,26.5579,54.8807,71.60,49,0,6.91,IR,1611820151
329,Jām Sāhib,26.2958,68.6292,77.34,15,0,3.62,PK,1611820164
339,Nizwá,22.9333,57.5333,76.03,19,0,5.68,OM,1611820166
352,Dalkola,25.8500,87.8500,77.77,29,0,1.61,IN,1611820168
370,Ikungi,-5.1333,34.7667,74.55,59,0,5.61,TZ,1611820171
499,Sittwe,20.1500,92.9000,77.25,42,0,9.06,MM,1611820191


In [ ]:
#* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

In [15]:
df_vacay_filter["Hotel Name"]= ""
df_vacay_filter

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,name,Latitude,Longitude,Temp_Max,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,Hermanus,-34.4187,19.2345,72.00,55,0,1.01,ZA,1611820090,
113,Ashta,23.0167,76.7167,78.35,23,0,5.41,IN,1611820131,
170,Hong Kong,22.2855,114.1577,73.00,63,0,9.31,HK,1611819952,
227,Bubaque,11.2833,-15.8333,77.92,54,0,6.11,GW,1611820149,
242,Bandar-e Lengeh,26.5579,54.8807,71.60,49,0,6.91,IR,1611820151,
329,Jām Sāhib,26.2958,68.6292,77.34,15,0,3.62,PK,1611820164,
339,Nizwá,22.9333,57.5333,76.03,19,0,5.68,OM,1611820166,
352,Dalkola,25.8500,87.8500,77.77,29,0,1.61,IN,1611820168,
370,Ikungi,-5.1333,34.7667,74.55,59,0,5.61,TZ,1611820171,
499,Sittwe,20.1500,92.9000,77.25,42,0,9.06,MM,1611820191,


In [ ]:
["Hotel Name"]= ""

ModuleNotFoundError: No module named 'config'

In [ ]:
# x = df["Latitude"].to_list()
# # x_north = []
# for num in x:
#     if num >= 0:
#         x_north.append(num)

# print(x_north)

        
#df.loc["Latitude" >= "1"]

In [ ]:
y = df["Humidity"].values

x = df["Latitude"].to_list()
x_north = []
for num in x:
    if num >= 0:
        x_north.append(num)
X = [[num] for num in x_north]


In [ ]:
df.to_csv("cities.csv")